In [7]:
pip install owlready2

Defaulting to user installation because normal site-packages is not writeable
  Using cached Owlready2-0.36.tar.gz (23.8 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for owlready2: filename=Owlready2-0.36-py3-none-any.whl size=19824048 sha256=41ae99c1c02afc0f487577bd909a968680101164e758bdafed4cf03ded0eb769
  Stored in directory: /home/pablo/.cache/pip/wheels/4a/4e/30/fb2dd09a5e7f330431cfc21beb1260f42de44e8a20d2d7b087
Successfully built owlready2
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install Levenshtein

In [232]:
pip install rdflib

     |████████████████████████████████| 482 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 259 kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ontospy[FULL] -U

Defaulting to user installation because normal site-packages is not writeable
  Using cached ontospy-1.9.9.4-py2.py3-none-any.whl (7.7 MB)
  Using cached pyfiglet-0.8.post1-py2.py3-none-any.whl (865 kB)
  Using cached SPARQLWrapper-1.8.5-py3-none-any.whl (26 kB)
  Using cached rdflib-6.1.1-py3-none-any.whl (482 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 1.2 MB/s eta 0:00:001.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached Pygments-2.1.3-py2.py3-none-any.whl (755 kB)
  Using cached asgiref-3.5.0-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 4.7 MB/s eta 0:00:00
  Using cached sqlparse-0.4.2-py3-none-any.whl (42 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
ERROR: pip's d

In [9]:
import sqlite3
import pandas as pd
import csv
import mysql.connector
import numpy as numpy
import json
import ontospy
from sqlite3 import Error
from owlready2 import *
from math import floor, ceil
from enum import Enum 

In [4]:
dataname = 'database.db'
memory = ':memory:'
try:
    con = sqlite3.connect(dataname)
    print("Connection is established:" + dataname)
except Error:
    print (Error)
        
def sql_table(con):
    cursorObj = con.cursor()
    sql_file = open("Agencia.sql")
    sql_as_string = sql_file.read()
    cursorObj.executescript(sql_as_string)
    con.commit()

#http://vocab.gtfs.org/gtfs.ttl
sql_table(con)

Connection is established:database.db


OperationalError: table AGENCY already exists

In [10]:
def connect_database():
    try:
        connection = mysql.connector.connect(host='localhost',
                                     user='root',
                                     password='gtfs',
                                     database='gtfs',
                                     charset='utf8mb4')

        if connection.is_connected():
                db_Info = connection.get_server_info()
                print("Connected to MySQL Server version ", db_Info)
                cursor = connection.cursor()
                cursor.execute("select database();")
                record = cursor.fetchone()
                print("You're connected to database: ", record)
            
        return connection, cursor

    except Error as e:
        print("Error while connecting to MySQL", e)
    #finally:
    #    if connection.is_connected():
    #        cursor.close()
    #        connection.close()
    #        print("MySQL connection is closed")

In [11]:
def get_tables(connection, cursor):
    cursor.execute("show tables;")
    myresult = cursor.fetchall()
    return myresult
def get_table_properties(table, cursor):
    sql_select_Query = "select * from " + table
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    # get all records
    records = cursor.fetchall()
    return cursor.column_names

In [ ]:


cur = connection.cursor()
#cur.execute("SELECT * FROM AGENCY")
#cur.fetchall()
cur.execute('DROP DATABASE IF EXISTS prueba.db')
df = pd.read_sql_query("SELECT * FROM AGENCY",con)
print(df.to_string())



In [ ]:
df = pd.read_csv('examples/datasets/sql/1/AGENCY.csv')
print(df.to_string()) 

In [6]:
def loadOntology(onto):
    my_world = World()
    ontology = onto
    onto = my_world.get_ontology(ontology).load()
    return onto, my_world

In [184]:
print(onto.base_iri)
print(onto.name)

NameError: name 'onto' is not defined

In [12]:
def get_ontology_classes(my_world):
    try:
        l = list(my_world.sparql("""
                   SELECT  DISTINCT ?x 
                   { ?x a rdfs:Class . }
            """))
    except:
        l = list(my_world.sparql("""
                   SELECT  DISTINCT ?x 
                   { ?x a owl:Class . }
            """))
    class_list_names = []
    classes = []
    for elem in l:
        terms = str(elem).split('.')
        class_list_names.append((re.sub(']', '',terms.pop())))
        classes.append(elem[0])
    return class_list_names, classes

def get_class_properties(my_world, class_name, prefix, uri):
    t = re.sub('\.', ':', str(class_name))
    l = list(my_world.sparql("""
               PREFIX """ + prefix + """: """ + uri 
               + """SELECT  DISTINCT ?p 
               { ?x a """ + t + """;
                    ?p ?o. }
        """))
    return l

def get_ontology_properties(my_world, prefix, uri):
    graph = my_world.as_rdflib_graph()
    try:
        l = list(graph.query_owlready("""
                   PREFIX """ + prefix + """: """ + uri
                   + """SELECT  DISTINCT ?p  
                   { ?x a rdfs:Class;
                        ?p ?o. }
            """))
    except Error as e:
        print(e)
    return l

In [14]:
def loadOntology(uri):
    return ontospy.Ontospy(uri, verbose=True, hide_implicit_preds=True, hide_implicit_types=True)

#List<Classes>
def get_onto_classes(ontology):
    classes = []
    for x in ontology.all_classes:
        #print("Nombre: ", x.qname, "Dominio: ", x.domain_of, "Range: ", x.range_of)
        #print("Nombre: ", x.qname, "triples: ", x.triples)
        #if (x.domain_of):
        #print(x.describe())
        classes.append(x)
    return classes

#List<Properties>
def get_class_properties(ontology, o_class):
    properties = []
    total_prop = []
    for x in ontology.all_properties:
        for y in x.ranges:
            if y.qname == o_class.qname:
                properties.append(x)
        for y in x.domains:
            if y.qname == o_class.qname:
                properties.append(x)
    total_prop.append([o_class, properties])
    return properties

#List<Class, List<Properties>>
def get_onto_properties(ontology, classes):
    properties = []
    for x in classes:
        properties_class = get_class_properties(ontology, x)
        if len(properties_class) != 0:
            properties.append(properties_class)
    return properties


def printTurtle(x):
    x.printSerialize(x)

In [15]:
connection, cursor = connect_database()
tables = get_tables(connection,cursor)

for y in tables:
        sql_select_Query = "select * from " + y[0]
        cursor = connection.cursor()
        cursor.execute(sql_select_Query)
        # get all records
        records = cursor.fetchall()
        print("Total number of columns in table: " + y[0], cursor.column_names)

#'http://vocab.gtfs.org/terms#' 'http://xmlns.com/foaf/0.1/'
ontology = loadOntology('http://vocab.gtfs.org/terms#')
print("\nReading  Ontology...\n")
oc_classes = get_onto_classes(ontology)
print("By library classes method: ", oc_classes)
oc_properties = get_onto_properties(ontology, oc_classes)



Connected to MySQL Server version  8.0.26
You're connected to database:  ('gtfs',)
Total number of columns in table: AGENCY ('agency_id', 'agency_name', 'agency_url', 'agency_timezone', 'agency_lang', 'agency_phone', 'agency_fare_url')
Total number of columns in table: CALENDAR ('service_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'start_date', 'end_date')
Total number of columns in table: CALENDAR_DATES ('service_id', 'date', 'exception_type')
Total number of columns in table: FEED_INFO ('feed_publisher_name', 'feed_publisher_url', 'feed_lang', 'feed_start_date', 'feed_end_date', 'feed_version')
Total number of columns in table: FREQUENCIES ('trip_id', 'start_time', 'end_time', 'headway_secs', 'exact_times')
Total number of columns in table: ROUTES ('route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color')
Total number of columns in table: SHAPES ('shape_id', 'shape

Reading: <http://vocab.gtfs.org/gtfs.ttl>
.. trying rdf serialization: <turtle>
..... success!
----------
Loaded 870 triples.
----------
RDF sources loaded successfully: 1 of 1.
..... 'http://vocab.gtfs.org/gtfs.ttl'
----------


Scanning entities...
----------
Ontologies.........: 1
Classes............: 29
Properties.........: 52
..annotation.......: 0
..datatype.........: 0
..object...........: 0
Concepts (SKOS)....: 0
Shapes (SHACL).....: 0
----------

Reading  Ontology...

By library classes method:  [<Class *http://www.w3.org/ns/dcat#Dataset*>, <Class *http://xmlns.com/foaf/0.1/Agent*>, <Class *http://vocab.gtfs.org/terms#Agency*>, <Class *http://vocab.gtfs.org/terms#CalendarDateRule*>, <Class *http://vocab.gtfs.org/terms#CalendarRule*>, <Class *http://vocab.gtfs.org/terms#DropOffType*>, <Class *http://vocab.gtfs.org/terms#FareClass*>, <Class *http://vocab.gtfs.org/terms#FareRule*>, <Class *http://vocab.gtfs.org/terms#Feed*>, <Class *http://vocab.gtfs.org/terms#Frequency*>, <Class *http://vocab.gtfs.org/terms#PaymentMethod*>, <Class *http://vocab.gtfs.org/terms#PickupType*>, <Class *http://vocab.gtfs.org/terms#Route*>, <Class *http://vocab.gtfs.org/terms#RouteType*>, <Class *http://vocab.gtfs.org/terms#Ser

In [42]:
class RMLEntity(object):
    def __init__(self, table, onto_class, class_properties):
        self.onto_class = onto_class
        self.onto_properties = class_properties
        self.table = table

In [49]:
def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    return distances[len(token1)][len(token2)]
    
#min(1, 1 - abs(t1-t2)/len(token1))

def string_distance(str1, str2):
    distances = numpy.zeros(len(str1))
    streak = 0
    for x in range(0, len(str1)):
        for y in range(x, len(str2)):
            char1 = str1[x].lower()
            char2 = str2[y].lower()
            if char1 == char2:
                val = min(1, 1 - abs(x-y)/(max(len(str1), len(str2))))
                #print(val)
                if distances[x] < val:
                    distances[x] = val + streak
                if val == 1:
                    streak += 0.1
                    break
    #print(distances)
    return numpy.sum(distances)/(max(len(str1), len(str2)))

def correspondenceClass(oc_classes, table):
    dis = 100
    term = ""
    for x in oc_classes:
        #print(x[0])
        term1 = str(table).replace('_', '').lower()
        term2 = x.qname.split(':').pop()
        #print("Term1: ", term1)
        #print("Term2: ", term2)
        auxDis = levenshteinDistanceDP(term1, term2)
        #print(auxDis)
        if auxDis < dis:
            dis = auxDis
            #properties = get_class_properties(ontology, x)
            #propertiesCorrespondence = getPropertiesCorrespondence(columns, properties)
            term = x
    return term

def getPropertiesCorrespondence(columns, properties):
    cor_properties = []
    for x in properties:
        dis = 100
        term = []
        for y in columns:
            term1 = str(y).replace('_', '').lower()
            term2 = x.qname.split(':').pop()
            auxDis = levenshteinDistanceDP(term1, term2)
            if auxDis < dis:
                dis = auxDis
                term = [x, y]
        cor_properties.append(term)
    return cor_properties

def getRMLEntities(tables, oc_classes, ontology, cursor):
    classes = []
    for x in range(0, len(tables)):
        columns = get_table_properties(tables[x][0], cursor)
        #classes.append([tables[x], correspondenceClass(oc_classes, tables[x], ontology, columns)])
        onto_class = correspondenceClass(oc_classes, tables[x])
        properties = get_class_properties(ontology, onto_class)
        onto_properties = getPropertiesCorrespondence(columns, properties)
        classes.append(RMLEntity(tables[x], onto_class, onto_properties))
    return classes



classCor = getRMLEntities(tables, oc_classes, ontology, cursor)
for x in classCor:
    print(x.onto_properties)
    print(x.onto_class)


[[<Property *http://vocab.gtfs.org/terms#agency*>, 'agency_id'], [<Property *http://vocab.gtfs.org/terms#fareUrl*>, 'agency_url']]
<Class *http://vocab.gtfs.org/terms#Agency*>
[]
<Class *http://vocab.gtfs.org/terms#CalendarRule*>
[[<Property *http://vocab.gtfs.org/terms#dateAddition*>, 'date']]
<Class *http://vocab.gtfs.org/terms#CalendarDateRule*>
[]
<Class *http://vocab.gtfs.org/terms#Feed*>
[[<Property *http://vocab.gtfs.org/terms#endTime*>, 'end_time'], [<Property *http://vocab.gtfs.org/terms#headwaySeconds*>, 'headway_secs'], [<Property *http://vocab.gtfs.org/terms#startTime*>, 'start_time'], [<Property *http://vocab.gtfs.org/terms#usesExactTimes*>, 'exact_times']]
<Class *http://vocab.gtfs.org/terms#Frequency*>
[[<Property *http://vocab.gtfs.org/terms#longName*>, 'route_long_name'], [<Property *http://vocab.gtfs.org/terms#route*>, 'route_id'], [<Property *http://vocab.gtfs.org/terms#routeType*>, 'route_type'], [<Property *http://vocab.gtfs.org/terms#shortName*>, 'route_short_name

In [10]:
def loadProperties(outputFile, propertiesFile):
    with open('properties/' + propertiesFile) as json_file:
        data = json.load(json_file)
        json_file.close()
    return data

def writeProperties(output, properties):
    with open('outputs/' + output, 'w') as fw:
        original_stdout = sys.stdout
        sys.stdout = fw
        for x in properties['list_prefixes']:
            print('@prefix ' + x['prefix'] + ": " + x['URI'])
        print('@base ' + properties['base']['URI'])
        sys.stdout = original_stdout
        fw.close()
#print(data['list_prefixes'])

def LogicalSource(table, SQLversion):
    level = 8
    print (" "*level, "rml:logicalSource [")
    print (" "*(2*level), "rml:source: " + '"' + "<#DB_source>" + '"')
    print (" "*(2*level), "rr:sqlVersion " + '"' + SQLversion + '"')
    print (" "*(2*level), "rr:tableName: " + '"' + table[0] + '"')
    print (" "*level, "];")
    
def TripleMap(subject):
    level = 8
    print ("<" + subject[0] + ">")
    print (" "*level, "a rr:TriplesMap;")
    #print ("\n")
    LogicalSource(subject, "SQL2008")
    
#def SubjectMap(ontologyClasses):
    

def deleteFileContent(file):
    file = open("outputs/" + file,"r+")
    file.truncate(0)
    file.close()
#def RMLTable(oc, table):

def RML_Transformation(output, properties, tables, ontologyClasses):
    data = loadProperties(output, properties)
    writeProperties(output, data)
    with open('outputs/' + output, 'w') as fw:
        original_stdout = sys.stdout
        sys.stdout = fw
        for x in tables:
            TripleMap(x)
        sys.stdout = original_stdout
        fw.close()
        
connection, cursor = connect_database()
tables = get_tables(connection,cursor)
onto = loadOntology("gtfs")
ontologyClasses = get_ontology_classes()
RML_Transformation('output.txt', 'properties.json', tables, ontologyClasses)

Connected to MySQL Server version  8.0.26
You're connected to database:  ('gtfs',)


NameError: name 'my_world' is not defined